In [1]:
import pandas as pd

import numpy as np

#Reading the required excel sheets and columns from the BI Daily Reports. The selected columns are considered valid for the required task
r1 = pd.read_excel('BI NG daily reports of 2022-06-16.xlsx', 'Contracts', usecols = ['Contract','generation','LTOperiod','LTOunits'])

r2 = pd.read_excel('BI NG daily reports of 2022-06-16.xlsx','Sps Contracts', usecols = ['Contract','activation_start','activation_end'])

r3 = pd.read_excel('BI NG daily reports of 2022-06-16.xlsx','Contracts per PaidPeriod', usecols = ['Contract','PaidPeriods'])

r4 = pd.read_excel('BI NG daily reports of 2022-06-16.xlsx','CRM contracts', names = ['Contract','CRMcontract'])

r5 = pd.read_excel('BI NG daily reports of 2022-06-16.xlsx','Last payments', usecols = ['Contract','payment_date','activation_end'])

r5.columns = ['Contract','last_payment_date','lastpayment_activation_end']

big_table = pd.read_csv("Bigtable_May'22.csv", usecols = ['Contract Id','Type','Created Date','Product Name','Customer Signed By: Full Name','Customer Signed By: Phone','Customer: Secondary Phone number'], encoding='windows-1252')

big_table.columns = ['Contract','Type','Created_Date','Product Name','Customer_Name','Customer_Phone_Number','Secondary_Phone_Number']

SG = pd.read_csv('segement.csv')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (32) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [2]:
r2['activation_start'] = pd.to_datetime(r2['activation_start']).dt.normalize()
r2['activation_end'] = pd.to_datetime(r2['activation_end']).dt.normalize()
r5['last_payment_date'] = pd.to_datetime(r5['last_payment_date']).dt.normalize()
r5['lastpayment_activation_end'] = pd.to_datetime(r5['lastpayment_activation_end']).dt.normalize()
big_table['Created_Date'] = pd.to_datetime(big_table['Created_Date'], format='%d/%m/%Y')

In [3]:
check1 = pd.merge(pd.merge(pd.merge(pd.merge(r1, r2, how='left', on='Contract'), r3, how='left', on='Contract'), r4, how='left', on='Contract'), r5, how='left', on='Contract') 

check1 = pd.merge(check1, big_table, how='left', left_on='CRMcontract', right_on='Contract')

check1 = pd.merge(check1, SG[['Contract Id','Potential Grouping Other']], how='left', left_on='CRMcontract', right_on='Contract Id')

check1 = check1[check1['Type'] == 'Production']

check1["ref_activation_end"] = check1[["activation_end","lastpayment_activation_end"]].max(axis=1)

check1['ref_activation_end'] = check1['ref_activation_end'].fillna('1900-01-01')

check1['ref_activation_end'] = pd.to_datetime(check1['ref_activation_end']).dt.normalize()

In [4]:
filters = [
   (check1['Product Name'].str.contains("NWO") == True)
]
values = ['LCP']

check1["lcp_status"] = np.select(filters, values, default='Non-LCP')

In [5]:
check1 = check1[((check1["light_days"] == 5) | (check1["light_days"] == 3) | (check1["light_days"] == 2) | (check1["light_days"] == 1)) & (check1["lcp_status"] != 'LCP') & (check1["Type"] == 'Production') & (check1["Potential Grouping Other"] == 'Good')]

KeyError: 'light_days'

In [ ]:
check1 = check1.drop(['Contract_x','LTOperiod','LTOunits','Product Name','Type','Contract_y','activation_start','activation_end','PaidPeriods','last_payment_date','lastpayment_activation_end','ref_activation_end','lcp_status'], axis=1)

In [ ]:
check1.groupby(['generation','light_days']).count()['CRMcontract']